In [1]:
import os
import random
import shutil
import json
from PIL import Image
import cv2
from ultralytics import YOLO
import pandas as pd

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 


Cambio de nombre de las imágenes

In [4]:
# Número actual de imágenes
imagenes_actuales = 125

# Ruta de la carpeta original que contiene las imágenes
ruta_original = "imagenes"

# Nombre de la nueva carpeta donde se guardarán las imágenes renombradas
nueva_carpeta = "Dataset/"

# Asegurarse de que la nueva carpeta existe o se crea
if not os.path.exists(nueva_carpeta):
    os.makedirs(nueva_carpeta)

# Recorrer las imágenes en la carpeta original
for i, nombre_archivo in enumerate(os.listdir(ruta_original)):
    # Verificar si el archivo es una imagen (puedes añadir más extensiones si lo necesitas)
    if nombre_archivo.endswith(('.jpg')):
        # Definir el nuevo nombre para la imagen (ejemplo: imagen_1.jpg)
        nuevo_nombre = f"imagen_{i+1+imagenes_actuales}.jpg"

        # Ruta completa del archivo original
        ruta_original_completa = os.path.join(ruta_original, nombre_archivo)

        # Ruta completa del nuevo archivo
        ruta_nueva_completa = os.path.join(nueva_carpeta, nuevo_nombre)

        # Copiar y renombrar la imagen a la nueva carpeta
        shutil.copy(ruta_original_completa, ruta_nueva_completa)

print("Proceso completado.")


Proceso completado.


Limpieza de los directorios train, val y test para añadir imagenes nuevamente

In [10]:
# Función para limpiar los directorios de imágenes y etiquetas
def limpiar_directorios(carpeta_destino):
    # Subcarpetas para los conjuntos de datos
    subcarpetas = ['train', 'test', 'val']
    
    # Tipos de archivo que queremos eliminar
    extensiones = ['.jpg', '.png', '.txt']  # Añade aquí otras extensiones de imagen si es necesario
    
    for subcarpeta in subcarpetas:
        carpeta_imagenes = os.path.join(carpeta_destino, subcarpeta, "images")
        carpeta_labels = os.path.join(carpeta_destino, subcarpeta, "labels")
        
        # Eliminar archivos en la carpeta de imágenes
        if os.path.exists(carpeta_imagenes):
            for archivo in os.listdir(carpeta_imagenes):
                if any(archivo.endswith(ext) for ext in extensiones):
                    os.remove(os.path.join(carpeta_imagenes, archivo))
        
        # Eliminar archivos en la carpeta de etiquetas
        if os.path.exists(carpeta_labels):
            for archivo in os.listdir(carpeta_labels):
                if archivo.endswith('.txt'):
                    os.remove(os.path.join(carpeta_labels, archivo))
    
    print("Los directorios de train, test y val han sido limpiados.")

# Ejemplo de uso
carpeta_destino = "VC_P4-NUM_PLATES"  # Ruta a la carpeta principal
limpiar_directorios(carpeta_destino)

Los directorios de train, test y val han sido limpiados.


Distribución de las imágenes entre train (70%), test (20%) y val (10%).

In [7]:
# Definir las clases y el ID de la clase "matricula"
clases = ["matricula"]

# Función para convertir al formato YOLO y escribir en el archivo .txt
def convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt):
    # Calcular coordenadas y tamaño del bounding box normalizado
    x_center = ((x_min + x_max) / 2) / imagen_ancho
    y_center = ((y_min + y_max) / 2) / imagen_alto
    width = (x_max - x_min) / imagen_ancho
    height = (y_max - y_min) / imagen_alto

    # Escribir en el archivo .txt en formato YOLO
    archivo_txt.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Ruta de la carpeta original que contiene las imágenes y JSON
carpeta_original = "Dataset"

# Carpeta de destino principal
carpeta_destino = "VC_P4-NUM_PLATES"

# Subcarpetas dentro de cada subconjunto
subcarpetas = ['train', 'test', 'val']
carpeta_images = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "images") for subcarpeta in subcarpetas}
carpeta_labels = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "labels") for subcarpeta in subcarpetas}

# Asegurarse de que las carpetas de destino existen o se crean
for subcarpeta in subcarpetas:
    os.makedirs(carpeta_images[subcarpeta], exist_ok=True)
    os.makedirs(carpeta_labels[subcarpeta], exist_ok=True)

# Obtener todas las imágenes de la carpeta original
imagenes = [f for f in os.listdir(carpeta_original) if f.endswith('.jpg')]

# Mezclar las imágenes de forma aleatoria
random.shuffle(imagenes)

# Calcular cuántas imágenes irán en cada subconjunto
total_imagenes = len(imagenes)
train_size = int(0.7 * total_imagenes)
test_size = int(0.2 * total_imagenes)
val_size = total_imagenes - train_size - test_size  # Lo que queda es para val

# Dividir las imágenes en tres subconjuntos
train_imagenes = imagenes[:train_size]
test_imagenes = imagenes[train_size:train_size + test_size]
val_imagenes = imagenes[train_size + test_size:]

# Función para copiar imágenes a las carpetas 'images' y crear archivos .txt con anotaciones en 'labels'
def copiar_imagenes_con_anotaciones(imagenes, carpeta_imagenes, carpeta_labels):
    for imagen in imagenes:
        # Copiar la imagen a la subcarpeta "images"
        ruta_origen_imagen = os.path.join(carpeta_original, imagen)
        ruta_destino_imagen = os.path.join(carpeta_imagenes, imagen)
        shutil.copy(ruta_origen_imagen, ruta_destino_imagen)

        # Generar la ruta del archivo JSON correspondiente
        nombre_sin_extension = os.path.splitext(imagen)[0]  # Nombre sin extensión
        archivo_json = f"{nombre_sin_extension}.json"
        ruta_origen_json = os.path.join(carpeta_original, archivo_json)

        # Crear el archivo .txt en la carpeta "labels" y convertir el JSON a formato YOLO, si existe el JSON
        ruta_txt = os.path.join(carpeta_labels, f"{nombre_sin_extension}.txt")
        if os.path.exists(ruta_origen_json):
            with open(ruta_txt, 'w') as archivo_txt, open(ruta_origen_json, 'r') as json_file:
                datos = json.load(json_file)

                # Abrir la imagen para obtener sus dimensiones
                with Image.open(ruta_destino_imagen) as img:
                    imagen_ancho, imagen_alto = img.size

                # Recorrer los objetos anotados en el JSON y escribir en formato YOLO
                for forma in datos['shapes']:
                    if forma['label'] == 'matricula' and forma['shape_type'] == 'rectangle':
                        puntos = forma['points']
                        x_min = min(p[0] for p in puntos)
                        y_min = min(p[1] for p in puntos)
                        x_max = max(p[0] for p in puntos)
                        y_max = max(p[1] for p in puntos)

                        # Llamar a la función que convierte al formato YOLO y escribe en el archivo .txt
                        class_id = clases.index('matricula')
                        convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt)

# Copiar las imágenes y crear los archivos .txt en las carpetas correspondientes
copiar_imagenes_con_anotaciones(train_imagenes, carpeta_images['train'], carpeta_labels['train'])
copiar_imagenes_con_anotaciones(test_imagenes, carpeta_images['test'], carpeta_labels['test'])
copiar_imagenes_con_anotaciones(val_imagenes, carpeta_images['val'], carpeta_labels['val'])

print("Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.")

Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.


Pruebas

In [1]:
# Importar las librerías necesarias
import cv2
import os
from ultralytics import YOLO

# Cargar los modelos entrenados
model_matriculas = YOLO('runs/detect/train/weights/best.pt')
model = YOLO('yolo11n.pt')

# Ruta del video de entrada
filename = "videoplayback.MP4"

# Abrir el archivo de video
cap = cv2.VideoCapture(filename)

# Obtener información sobre el video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Crear la ruta de salida para el video procesado
output_filename = 'video_con_detecciones_2.mp4'
output_path = os.path.join(os.getcwd(), output_filename)

# Crear un VideoWriter para guardar el video con detecciones
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Procesar el video frame a frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Si no hay más frames, salir del bucle

    # Realizar detecciones en el frame actual
    results = model_matriculas(frame)

    # Obtener el frame anotado con las detecciones
    annotated_frame = results[0].plot()

    # Guardar el frame anotado en el archivo de video de salida
    out.write(annotated_frame)
    
    # Mostrar el frame anotado en una ventana
    cv2.imshow('Video', annotated_frame)
    
    # Permitir salir al presionar 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar los recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video guardado en {output_path}")


WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 

0: 384x640 (no detections), 56.5ms
Speed: 3.5ms preprocess, 56.5ms inference, 32.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 3.5ms preprocess, 10.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 2.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.5ms
Speed: 2.5ms preprocess, 14.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 1.5ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 2.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 1.5ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 

Tarea: Detector de Personas, coches y matrículas. 

In [1]:
from collections import defaultdict
import numpy as np
import cv2
import torch
import math
from ultralytics import YOLO
import easyocr
import csv
import os

# Initialize YOLO and EasyOCR models
model_detection = YOLO('yolo11n.pt')  # Model for detecting people and cars
model_license_plate = YOLO('runs/detect/train/weights/best.pt')  # License plate detection model
reader = easyocr.Reader(['en'])  # EasyOCR for license plate recognition

# Class labels
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus"]

# Initialize counters and sets to track unique detections
counts = {'person': 0, 'car': 0, 'plate': 0, 'bus':0}
seen_ids = {'person': set(), 'car': set(), 'plate': set(), 'bus':set()}

# Output CSV file for detection details
csv_file = 'resultados_detalles.csv'
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Tipo_objeto', 'Confianza', 'ID_Tracking', 'x1', 'y1', 'x2', 'y2',
                         'Matrícula', 'Confianza_Matrícula', 'mx1', 'my1', 'mx2', 'my2', 'Texto_Matrícula'])

# Capture video
vid = cv2.VideoCapture("C0142.MP4")
frame_width, frame_height = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))

# Initialize video writer for output video
output_video = 'output_video_with_results.mp4'
output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_number = 0

while True:
    ret, img = vid.read()
    if not ret:
        break
    
    frame_number += 1

    # Detect people and cars
    results = model_detection.track(img, persist=True, classes=[0, 2, 5])  # Detect classes 0 (person), 2 (car), and 5 (bus)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            cls = int(box.cls[0])  # Class label
            confidence = float(box.conf[0])  # Confidence score
            object_type = classNames[cls] if cls < len(classNames) else 'unknown'

            # Tracking ID
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ''

            # Check if the object has already been counted
            if track_id and track_id not in seen_ids[object_type]:
                # Count only new appearances
                counts[object_type] += 1
                seen_ids[object_type].add(track_id)  # Add the ID to the set of seen IDs

            if object_type in ["car", "bus"]:
                # Extract the car region for license plate detection
                car_img = img[y1:y2, x1:x2]
                plate_results = model_license_plate(car_img)

                # Draw plate detections on the image
                for plate in plate_results:
                    
                    plate_boxes = plate.boxes
                    for pbox in plate_boxes:
                        px1, py1, px2, py2 = map(int, pbox.xyxy[0])

                        # Crop and preprocess license plate image for OCR
                        plate_img = car_img[py1:py2, px1:px2]
                        if plate_img.size == 0:
                            continue
                        
                        if track_id and track_id not in seen_ids['plate']:
                        # Count only new appearances
                            counts['plate'] += 1
                            seen_ids['plate'].add(track_id)  # Add the ID to the set of seen IDs

                        # Resize and apply CLAHE and thresholding
                        scale_factor = 1.5
                        resized_plate = cv2.resize(plate_img, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
                        gray_plate = cv2.cvtColor(resized_plate, cv2.COLOR_BGR2GRAY)
                        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
                        contrast_plate = clahe.apply(gray_plate)
                        binary_plate_adaptive = cv2.adaptiveThreshold(contrast_plate, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

                        # Run OCR on the processed plate image
                        try:
                            result = reader.readtext(
                                binary_plate_adaptive,
                                decoder='beamsearch',
                                detail=1,
                                allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
                            )

                            if result:
                                for (bbox, text, prob) in result:
                                    # Log results in CSV and draw on frame
                                    with open(csv_file, mode='a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow([frame_number, object_type, f'{confidence:.2f}', track_id,
                                                         x1, y1, x2, y2, text, f'{prob:.2f}', 
                                                         px1, py1, px2, py2, text])

                                    # Draw detected plate text on image
                                    cv2.putText(img, text, (x1 + px1, y1 + py1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                                    cv2.rectangle(img, (x1 + px1, y1 + py1), (x1 + px2, y1 + py2), (0, 255, 0), 2)
                        except Exception as e:
                            print("Error during OCR:", e)

            # Draw bounding boxes for all objects
            color = (0, 0, 255) if object_type == "person" else (255, 0, 0)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f'{object_type} {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Update frame count overlay
    cv2.putText(img, f'Frame: {frame_number}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(img, f'People: {counts["person"]}, Cars: {counts["car"]}, Plates: {counts["plate"]}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Write annotated frame to output video
    output_writer.write(img)
    
    # Display the video
    cv2.imshow('Video with Detections', img)
    if cv2.waitKey(20) & 0xFF == 27:
        break

# Cleanup
vid.release()
output_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_video}")


WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 

0: 384x640 1 car, 1 bus, 39.0ms
Speed: 4.5ms preprocess, 39.0ms inference, 92.0ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 (no detections), 49.0ms
Speed: 4.5ms preprocess, 49.0ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 50.0ms
Speed: 3.0ms preprocess, 50.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 1 bus, 10.0ms
Speed: 2.5ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 1 matricula, 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 1 bus, 18.0ms
Speed: 3.0ms preprocess, 18.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: